In [1]:
import pandas as pd
# import pandas_profiling as pp

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src import util

In [4]:
util.set_context("ieee_fraud_detection")

Files in data directory:
______

ieee_fraud_detection/
    raw/
        train_transaction.csv.zip
        train_identity.csv.zip
        test_transaction.csv.zip
        test_identity.csv.zip
        sample_submission.csv.zip
        train_transaction.csv
        train_identity.csv
        test_transaction.csv
        test_identity.csv
        sample_submission.csv
______



In [5]:
tr_trx = pd.read_csv(util.get_path("raw/train_transaction.csv"))
tr_trx = util.reduce_mem_usage(tr_trx)
tr_trx.shape

100%|██████████| 394/394 [02:48<00:00,  2.35it/s]


Mem. usage decreased to 542.35 Mb (69.4% reduction)


(590540, 394)

In [7]:
te_trx = pd.read_csv(util.get_path("raw/test_transaction.csv"))
te_trx = util.reduce_mem_usage(te_trx)
te_trx.shape

100%|██████████| 393/393 [02:38<00:00,  2.48it/s]


Mem. usage decreased to 472.59 Mb (68.9% reduction)


(506691, 393)

In [8]:
tr_id = pd.read_csv(util.get_path("raw/train_identity.csv"))
te_id = pd.read_csv(util.get_path("raw/test_identity.csv"))
tr_id = util.reduce_mem_usage(tr_id)
te_id = util.reduce_mem_usage(te_id)
tr_id.shape, te_id.shape

100%|██████████| 41/41 [00:00<00:00, 261.69it/s]

Mem. usage decreased to 25.86 Mb (42.7% reduction)
Mem. usage decreased to 25.44 Mb (42.7% reduction)


((144233, 41), (141907, 41))

In [9]:
tr_trx.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
tr_id.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [11]:
tr_id.TransactionID.is_unique

True

In [12]:
col_overlap = set(tr_trx.columns).intersection(tr_id.columns)
col_overlap

{'TransactionID'}

In [13]:
col_diff = set(tr_trx.columns).difference(te_trx.columns)
assert len(col_diff) == 1
target_col = col_diff.pop()
target_col

'isFraud'

In [14]:
tr_trx["test_"] = False
te_trx["test_"] = True

In [15]:
tr = tr_trx.merge(tr_id, how="left", validate="1:1")
te = te_trx.merge(te_id, how="left", validate="1:1")
tr.shape, te.shape

((590540, 435), (506691, 434))

In [16]:
df = pd.concat([tr, te], sort=True)
df.shape

(1097231, 435)

In [17]:
# df.info(memory_usage="deep")

In [18]:
obj_cols = df.select_dtypes("object").columns.tolist()
for col in obj_cols:
    print(f"{col}: {df[col].nunique()} unique values")
    df[col] = df[col].astype("category")

DeviceInfo: 2799 unique values
DeviceType: 2 unique values
M1: 2 unique values
M2: 2 unique values
M3: 2 unique values
M4: 3 unique values
M5: 2 unique values
M6: 2 unique values
M7: 2 unique values
M8: 2 unique values
M9: 2 unique values
P_emaildomain: 60 unique values
ProductCD: 5 unique values
R_emaildomain: 60 unique values
card4: 4 unique values
card6: 4 unique values
id_12: 2 unique values
id_15: 3 unique values
id_16: 2 unique values
id_23: 3 unique values
id_27: 2 unique values
id_28: 2 unique values
id_29: 2 unique values
id_30: 87 unique values
id_31: 172 unique values
id_33: 461 unique values
id_34: 4 unique values
id_35: 2 unique values
id_36: 2 unique values
id_37: 2 unique values
id_38: 2 unique values


In [19]:
df.describe()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32,isFraud
count,1097228.0,1097228.0,1097228.0,1097228.0,1092483.0,1097228.0,1097228.0,1097228.0,1097228.0,1097228.0,...,95988.0,275224.0,274894.0,10218.0,1.023100e+04,9487.000000,10171.000,10210.0000,148257.0,590540.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,inf,inf,inf,inf,inf,NaN,0.034990
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,inf,6.304688e+00,2.835938,92.125,32.0625,0.0,0.183755
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,100.0,100.0,100.0,1.000000e+01,10.000000,100.000,100.0000,0.0,0.000000
25%,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,13.0,266.0,256.0,252.0,1.400000e+01,11.000000,321.000,121.0000,24.0,0.000000
50%,1.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,...,15.0,337.0,472.0,277.0,1.400000e+01,11.000000,321.000,147.0000,24.0,0.000000
75%,3.0,0.0,2.0,0.0,13.0,2.0,3.0,0.0,0.0,1.0,...,15.0,427.0,539.0,711.0,1.400000e+01,15.000000,360.000,173.7500,32.0,0.000000
max,4684.0,3256.0,3188.0,3188.0,2918.0,1429.0,5692.0,31.0,2252.0,376.0,...,29.0,671.0,661.0,854.0,4.400000e+01,26.000000,549.000,216.0000,48.0,1.000000


In [20]:
# pp.ProfileReport(df)

In [22]:
df.to_pickle(util.get_path("comb.pkl"))